# SPARK RDD part 1

In [1]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName="RDDstart")

### Tworzenie RDD

#### Lokalna kolekcja

W realnych zastosowaniach RDD tworzy się poprzez wczytanie danych z rozproszonych zasobów sieciowych takich jak na przykład HDFS, Cassandra, MongoDB, Amazon S3 i wielu innych. Mając doczynienia z dużo mniejszymi zbiorami, możemy je zaczytać w programie sterownika i przekształcić programowo w rozproszony zbiór RDD. Pomoże nam w tym funkcja parallelize. 

In [2]:
RDDlist = sc.parallelize(list(range(10)))

In [3]:
RDDlist

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [4]:
RDDlist.collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [5]:
RDDlist.take(5)

[0, 1, 2, 3, 4]

**_UWAGA: `collect` i `take` to niebezpieczne operacje - mogą doprowadzić do zapchania drivera i przerwania działania aplikacji_**

#### Plik

In [6]:
RDDfile = sc.textFile("RDD.txt")
RDDfile

RDD.txt MapPartitionsRDD[4] at textFile at DirectMethodHandleAccessor.java:104

In [7]:
RDDfile.collect()

['0,1,2,3,4,5,6,7,8,9']

In [8]:
RDDfile.take(5)

['0,1,2,3,4,5,6,7,8,9']

#### Inne RDD

In [9]:
RDDother = RDDfile.flatMap(lambda x: x.split(","))

In [10]:
RDDother.collect()

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [11]:
RDDother.take(5)

['0', '1', '2', '3', '4']

### Transformacje

##### map(fun)

Zwraca nowe RDD po zastosowaniu podanej funkcji na każdym elemencie oryginalnego RDD

In [12]:
RDDmap1 = RDDother.map(lambda x: int(x))
#Powyższe transformacje "wykonały się", ale nic się jeszcze nie stało. Nie dostaliśmy żadnego wyniku.
#Dzieje się tak ponieważ transformacje są leniwe. Dzięki temu można stworzyć cały graf przetwarzania przed 
#jego uruchomieniem.
RDDmap1.collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [13]:
RDDmap1.map(lambda x: x+1).collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [14]:
RDDmap1.map(lambda x: (x,1)).collect()

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1)]

In [15]:
RDDlist2 = sc.parallelize([list(range(5)),list(range(5,10))])

In [16]:
RDDlist2.collect()

[[0, 1, 2, 3, 4], [5, 6, 7, 8, 9]]

In [17]:
RDDlist2.map(sum).collect()

[10, 35]

In [18]:
RDDlist2.map(lambda x: [y + 1 for y in x]).collect()

[[1, 2, 3, 4, 5], [6, 7, 8, 9, 10]]

> **TODO**: Oblicz sumę całkowitą RDDlist2

In [19]:
sum(RDDlist2.map(sum).collect())


45

##### flatMap(func)

Zwraca nowe RDD po zastosowaniu podanej funkcji na każdym elemencie oryginalnego RDD oraz spłaszczeniu rezultatu

In [20]:
RDDlist2.flatMap(lambda x: x).collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [21]:
RDDlist2.flatMap(lambda x: x*2).collect()

[0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 5, 6, 7, 8, 9]

In [22]:
RDDlist2.flatMap(lambda x: [y*2 for y in x]).collect()

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

In [23]:
RDDlist3 = sc.parallelize([[[[0],[1]],[[2],[3]]],[[[4],[5]],[[6],[7]]]])
RDDlist3.collect()

[[[[0], [1]], [[2], [3]]], [[[4], [5]], [[6], [7]]]]

In [24]:
RDDlist3.flatMap(lambda x: x).collect()

[[[0], [1]], [[2], [3]], [[4], [5]], [[6], [7]]]

In [25]:
RDDlist3.flatMap(lambda x: x).flatMap(lambda x: x).collect()

[[0], [1], [2], [3], [4], [5], [6], [7]]

In [26]:
RDDlist3.flatMap(lambda x: x).flatMap(lambda x: x).flatMap(lambda x: x).collect()

[0, 1, 2, 3, 4, 5, 6, 7]

> **TODO**: Osiągnij taki sam wynik jak w komórce powyżej używając na RDDlist3: 1 x map i 2 x flatMap

In [27]:
RDDlist3.flatMap(lambda x: x ).flatMap(lambda x: x ).map(sum).collect()

[0, 1, 2, 3, 4, 5, 6, 7]

##### mapValues(func)

Zwraca nowe RDD po zastosowaniu podanej funkcji na każdej wartości oryginalnego RDD zawierającego pary klucz-wartość

In [28]:
RDDpair = sc.parallelize([("A",["Adam","Ada","Adrian"]),("B",["Bonifacy","Barnaba"])])
RDDpair.collect()

[('A', ['Adam', 'Ada', 'Adrian']), ('B', ['Bonifacy', 'Barnaba'])]

In [29]:
RDDpair.mapValues(lambda val: len(val)).collect()

[('A', 3), ('B', 2)]

In [30]:
RDDpair.mapValues(lambda val: len(val)).mapValues(lambda val: val*2).collect()

[('A', 6), ('B', 4)]

In [31]:
RDDpair.mapValues(lambda val: len(val)*2).collect()

[('A', 6), ('B', 4)]

Te same rezultaty można osiągnąć używając `map`, jednak jest to mniej wygodne

In [32]:
RDDpair.map(lambda x: (x[0],len(x[1]))).collect()

[('A', 3), ('B', 2)]

In [33]:
RDDpair.map(lambda x: (x[0],len(x[1]))).map(lambda x: (x[0],x[1]*2)).collect()

[('A', 6), ('B', 4)]

In [34]:
RDDpair.map(lambda x: (x[0],len(x[1]*2))).collect()

[('A', 6), ('B', 4)]

> **TODO**: Zmodyfikuj wartości w RDDpair tak aby zawierały nie imiona a liczby liter w imionach, następnie zsumuj liczby liter, wykonaj zadanie w dwóch wariantach - używając tylko `mapValues` i tylko `map`

In [39]:
RDDpair.mapValues(lambda val: [len(y) for y in val]).collect()

[('A', [4, 3, 6]), ('B', [8, 7])]

In [40]:
RDDpair.mapValues(lambda val: [len(y) for y in val]).mapValues(sum).collect()

[('A', 13), ('B', 15)]

In [41]:
RDDpair.map(lambda x: (x[0],[len(y) for y in x[1]])).collect()

[('A', [4, 3, 6]), ('B', [8, 7])]

In [42]:
RDDpair.map(lambda x: (x[0],[len(y) for y in x[1]])).map(lambda x: (x[0], sum(x[1]))).collect()

[('A', 13), ('B', 15)]

##### flatMapValues(func)

Zwraca nowe RDD po zastosowaniu podanej funkcji na każdej wartości oryginalnego RDD zawierającego pary klucz-wartość oraz spłaszczeniu rezultatu

In [43]:
RDDpair.flatMapValues(lambda x: x).collect()

[('A', 'Adam'),
 ('A', 'Ada'),
 ('A', 'Adrian'),
 ('B', 'Bonifacy'),
 ('B', 'Barnaba')]

In [36]:
RDDpair.flatMapValues(lambda x: [y.lower() for y in x]).collect()

[('A', 'adam'),
 ('A', 'ada'),
 ('A', 'adrian'),
 ('B', 'bonifacy'),
 ('B', 'barnaba')]

> **TODO**: Na podstawie RDDpair stwórz RDD o następującej strukturze: (litera, (imię, l. liter))

In [54]:
RDDpair.flatMapValues(lambda x: [(y,len(y)) for y in x]).collect()

[('A', ('Adam', 4)),
 ('A', ('Ada', 3)),
 ('A', ('Adrian', 6)),
 ('B', ('Bonifacy', 8)),
 ('B', ('Barnaba', 7))]

In [48]:
RDDpair1= RDDpair.flatMapValues(lambda x: x)

In [53]:
RDDpair1.map(lambda x: (x[0],(x[1],sum([len(y) for y in x[1]])))).collect()

[('A', ('Adam', 4)),
 ('A', ('Ada', 3)),
 ('A', ('Adrian', 6)),
 ('B', ('Bonifacy', 8)),
 ('B', ('Barnaba', 7))]

##### keys(), values()

Metody te tworzą nowe RDD odpowiednio z kluczy i wartości oryginalnego RDD (klucz, wartość)

In [60]:
RDDpair.values().collect()

[['Adam', 'Ada', 'Adrian'], ['Bonifacy', 'Barnaba']]

In [61]:
RDDpair.keys().collect()

['A', 'B']

> **TODO**: Na podstawie RDDpair stwórz RDD o następującej strukturze: (imię, l. liter)

In [65]:
RDDpair.flatMapValues(lambda x: [(y, len(y)) for y in x]).values().collect()

[('Adam', 4), ('Ada', 3), ('Adrian', 6), ('Bonifacy', 8), ('Barnaba', 7)]

##### filter(func)

Zwraca nowe RDD zawierające jedynie elementy które spełniają predykat

In [66]:
RDDmap1.filter(lambda x: x > 3).collect()

[4, 5, 6, 7, 8, 9]

In [67]:
RDDpair.filter(lambda x: x[0] == "A").collect()

[('A', ['Adam', 'Ada', 'Adrian'])]

Dobrą praktyką optymalizującą działanie programu jest filtrowanie RDD możliwie jak najwcześniej. Z poniższych dwóch komórek z kodem to druga jest bardziej optymalna.

In [68]:
RDDpair.flatMapValues(lambda x: x).filter(lambda x: x[0] != "B").collect()

[('A', 'Adam'), ('A', 'Ada'), ('A', 'Adrian')]

In [69]:
RDDpair.filter(lambda x: x[0] != "B").flatMapValues(lambda x: x).collect()

[('A', 'Adam'), ('A', 'Ada'), ('A', 'Adrian')]

> **TODO**: Odfiltruj parzyste liczby z RDDmap1

In [70]:
RDDmap1.filter(lambda x: x%2==0).collect()

[0, 2, 4, 6, 8]

##### join(RDD)

Zwraca RDD zawierające pary elementów z identycznymi kluczami w łączonych RDD.

In [71]:
RDDpair.join(RDDpair.flatMapValues(lambda x: x)).collect()

[('B', (['Bonifacy', 'Barnaba'], 'Bonifacy')),
 ('B', (['Bonifacy', 'Barnaba'], 'Barnaba')),
 ('A', (['Adam', 'Ada', 'Adrian'], 'Adam')),
 ('A', (['Adam', 'Ada', 'Adrian'], 'Ada')),
 ('A', (['Adam', 'Ada', 'Adrian'], 'Adrian'))]

In [72]:
RDDpair.collect()

[('A', ['Adam', 'Ada', 'Adrian']), ('B', ['Bonifacy', 'Barnaba'])]

In [73]:
RDDpair.flatMapValues(lambda x: x).collect()

[('A', 'Adam'),
 ('A', 'Ada'),
 ('A', 'Adrian'),
 ('B', 'Bonifacy'),
 ('B', 'Barnaba')]

##### union(RDD)

Zwraca RDD wynikłe z połączenia dwóch RDD

In [74]:
RDDmap1.union(RDDlist).collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

> **TODO**: Połącz dwa RDD powstałe na podstawie RDDpair o strukturze: (imię, l. liter) oraz (litera, imię), w wynikowym RDD mają znaleźć się jedynie obiekty których klucz zaczyna się na "B"

In [78]:
RDDpair.flatMapValues(lambda x: x).join(RDDpair.flatMapValues(lambda x: [(y, len(y)) for y in x]).values()).collect()

[]

In [82]:
RDDpair.filter(lambda x: x[0] == "B").flatMapValues(lambda x: x)\
.union(RDDpair.filter(lambda x: x[0]== "B").flatMapValues(lambda x: [(y, len(y)) for y in x]).values()).collect()

[('B', 'Bonifacy'), ('B', 'Barnaba'), ('Bonifacy', 8), ('Barnaba', 7)]

##### distinct()

Zwraca RDD zawierające jedynie unikalne wartości z oryginalnego RDD

In [47]:
RDDmap1.union(RDDlist).collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [48]:
RDDmap1.union(RDDlist).distinct().collect()

[0, 6, 1, 7, 2, 8, 3, 9, 4, 5]

##### groupBy(func)

Zwraca RDD z pogrupowanymi elementami

In [49]:
RDDlist.groupBy(lambda x: x % 2).collect()

[(0, <pyspark.resultiterable.ResultIterable at 0x22669c277f0>),
 (1, <pyspark.resultiterable.ResultIterable at 0x22669c27490>)]

In [50]:
RDDlist.groupBy(lambda x: x % 2).map(lambda x: (x[0], list(x[1]))).collect()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [51]:
RDDlist.groupBy(lambda x: x % 2).map(lambda x: (x[0], [y*3 for y in x[1]])).collect()

[(0, [0, 6, 12, 18, 24]), (1, [3, 9, 15, 21, 27])]

In [84]:
RDDpair2 = sc.parallelize([('Adam', 4), ('Ada', 3), ('Adrian', 6), ('Bonifacy', 8), ('Barnaba', 7)])

In [87]:
RDDpair2.groupBy(lambda x: "l.l.>4" if x[1] > 4 else "l.l.<=4").map(lambda x: (x[0], list(x[1]))).collect()

[('l.l.<=4', [('Adam', 4), ('Ada', 3)]),
 ('l.l.>4', [('Adrian', 6), ('Bonifacy', 8), ('Barnaba', 7)])]

> **TODO**: Pogrupuj RDDpair2 ze względu na pierwszą literę imienia

In [93]:
RDDpair2.groupBy(lambda x: "A" if x[0].startswith("A") else "B").mapValues(list).collect()

[('A', [('Adam', 4), ('Ada', 3), ('Adrian', 6)]),
 ('B', [('Bonifacy', 8), ('Barnaba', 7)])]

In [94]:
RDDpair2.groupBy(lambda x: x[0][0]).mapValues(list).collect()

[('A', [('Adam', 4), ('Ada', 3), ('Adrian', 6)]),
 ('B', [('Bonifacy', 8), ('Barnaba', 7)])]

##### groupByKey()

Zwraca RDD z wartościami pogrupowanymi w pojedynczą sekwencję dla każdego klucza. (_Jeśli grupowanie wykonywane jest w celu przeprowadzenia agregacji dla każdego klucza, optymalniejsze będzie wykorzystanie `reduceByKey` lub `aggregateByKey`._)

In [95]:
RDDpair3 = sc.parallelize([('A', 'Adam'),('A', 'Ada'),('A', 'Adrian'),('B', 'Bonifacy'),('B', 'Barnaba')])

In [96]:
RDDpair3.groupByKey().collect()

[('A', <pyspark.resultiterable.ResultIterable at 0x162f68c2f10>),
 ('B', <pyspark.resultiterable.ResultIterable at 0x162f68c6250>)]

In [97]:
RDDpair3.groupByKey().mapValues(list).collect()

[('A', ['Adam', 'Ada', 'Adrian']), ('B', ['Bonifacy', 'Barnaba'])]

> **TODO**: Na podstawie RDDpair stwórz RDD o następującej strukturze: (litera, (imię, l. liter)), następnie pogrupuj je po literze (zamień nowe wartości na listę tak aby były czytelne po użyciu `collect`)

In [100]:
RDDpair.flatMapValues(lambda x: [(y, len(y)) for y in x]).groupByKey().mapValues(list).collect()

[('A', [('Adam', 4), ('Ada', 3), ('Adrian', 6)]),
 ('B', [('Bonifacy', 8), ('Barnaba', 7)])]

##### reduceByKey(func)

Zwraca RDD z połączonymi wartościami dla każdego klucza. Funkcja redukująca musi być asocjacyjna _[(a x b) x c = a x (b x c)]_ i przemienna _[a x b = b x a]_. 

In [102]:
RDDpair4 = RDDpair3.mapValues(len)
RDDpair4.collect()

[('A', 4), ('A', 3), ('A', 6), ('B', 8), ('B', 7)]

In [103]:
RDDpair4.reduceByKey(lambda x,y: x+y).collect()

[('A', 13), ('B', 15)]

> **TODO**: Uzyskaj iloczyn dla każdego klucza w RDDpair4

In [104]:
RDDpair4.reduceByKey(lambda x,y: x*y).collect()

[('A', 72), ('B', 56)]

##### aggregateByKey(zeroValue, seqFunc, combFunc)

Rozbudowana wersja `reduceByKey` pozwalająca na zwrócenie wartości o innym typie niż oryginalne. Konieczne jest podanie trzech parametrów:
- zeroValue - domyślna wartość neutralna dla agregacji (dodawanie: 0, mnożenie: 1, tworzenie zbioru unikatowych wartości: pusty zbiór, itd.),
- seqFunc - funkcja agregująca wartości w oryginalnym RDD, przyjmuje dwa parametry, gdzie drugi jest włączany (dodawany itp.) do pierwszego
- combFunc - funkcja łącząca wartości uzyskane z pierwszej funkcji dla kluczy

In [105]:
RDDpair3.collect()

[('A', 'Adam'),
 ('A', 'Ada'),
 ('A', 'Adrian'),
 ('B', 'Bonifacy'),
 ('B', 'Barnaba')]

In [106]:
RDDpair3.aggregateByKey(0, (lambda acc,x: acc+len(x)), (lambda acc1,acc2: acc1+acc2)).collect()

[('A', 13), ('B', 15)]

In [107]:
RDDpair3.glom().collect()

[[('A', 'Adam')],
 [('A', 'Ada')],
 [('A', 'Adrian')],
 [('B', 'Bonifacy'), ('B', 'Barnaba')]]

Co się wydarzyło powyżej?

|RDDpair3 | seqFunc | seqFunc out. | combFunc | combFunc out. |
|:--------|:--------|:-------------|:---------|:------------- |
|('A', 'Adam') => | ('A', 0 + len('Adam')) => | ('A', 4) |  |  |
|('A', 'Ada') => | ('A', 0 + len('Ada')) => | ('A', 3) |  |  |
|('A', 'Adrian') => | ('A', 0 + len('Adrian')) => | ('A', 6) => | ('A', 4 + 3 + 6) => | ('A', 13) |
|('B', 'Bonifacy') => | ('B', 0 + len('Bonifacy')) => | ('B', 8) => | ('B', 8 + 7) => | ('B', 15) |
|('B', 'Barnaba') => | ('B', 0 + len('Barnaba')) => | ('B', 7) |  |  |

In [108]:
RDDpair3.aggregateByKey((0.,0.), (lambda acc,x: (acc[0]+len(x),acc[1]+1)), \
                        (lambda acc1,acc2: (acc1[0]+acc2[0],acc1[1]+acc2[1]))).collect()

[('A', (13.0, 3.0)), ('B', (15.0, 2.0))]

In [109]:
RDDpair3.aggregateByKey((0.,0.), (lambda acc,x: (acc[0]+len(x),acc[1]+1)), \
                        (lambda acc1,acc2: (acc1[0]+acc2[0],acc1[1]+acc2[1]))).mapValues(lambda x: x[0]/x[1]).collect()

[('A', 4.333333333333333), ('B', 7.5)]

> **TODO**: Z RDDpair3 uzyskaj RDD (klucz, iloczyn długości imion)

In [110]:
RDDpair3.aggregateByKey(1, (lambda acc,x: acc*len(x)), (lambda acc1,acc2: acc1*acc2)).collect()

[('A', 72), ('B', 56)]

### Akcje

##### collect()

Zwraca elementy zbioru na driver.

In [64]:
RDDlist.collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

##### collectAsMap()

Zwraca RDD (K, V) jako słownik.

In [65]:
RDDpair2.collectAsMap()

{'Adam': 4, 'Ada': 3, 'Adrian': 6, 'Bonifacy': 8, 'Barnaba': 7}

##### take(n)

Zwraca `n` pierwszych elementów zbioru na driver.

In [66]:
RDDlist.take(2)

[0, 1]

##### takeSample(withReplacement, num)

Zwraca losową próbę `num` elementów ze zwracaniem lub bez.

In [112]:
RDDlist.takeSample(True,12)

[5, 0, 2, 2, 8, 2, 2, 5, 2, 6, 2, 4]

In [113]:
RDDlist.takeSample(False, 5)

[5, 0, 9, 6, 8]

##### takeOrdered(n, [key])

Zwraca `n` pierwszych elementów zbioru stosując naturalny porządek lub inny wskazany.

In [69]:
RDDlist.takeOrdered(5)

[0, 1, 2, 3, 4]

In [70]:
RDDlist.takeOrdered(5, (lambda x: -x))

[9, 8, 7, 6, 5]

##### first()

Zwraca pierwszy element zbioru. Podobne do `take(1)`.

In [71]:
RDDlist.first()

0

##### count()

Zwraca liczbę elementów w zbiorze.

In [72]:
RDDlist.count()

10

##### sum()

Zwraca sumę elementów w RDD

In [73]:
RDDlist.sum()

45

##### countByKey()

Dla RDD (K, V) zwraca słownik (hashmap) typu (K, Int) z liczbą wystąpień kluczy.

In [74]:
RDDpair4.countByKey()

defaultdict(int, {'A': 3, 'B': 2})

In [75]:
RDDpair4.countByKey()["A"]

3

##### saveAsTextFile(path)

Zapisuje elementy zbioru do pliku (plików) tekstowych w podanym katalogu. Spark wywoła metodę `toString` na każdym elemencie RDD aby przekształcić je na linijkę teksu w pliku.

In [76]:
RDDlist.saveAsTextFile("...")

Py4JJavaError: An error occurred while calling o886.saveAsTextFile.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory file:/C:/Users/Karolina/Big Data/RDD_st/... already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.assertConf(SparkHadoopWriter.scala:299)
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:71)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopDataset$1(PairRDDFunctions.scala:1091)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:406)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1089)
	at org.apache.spark.rdd.P

##### reduce(func)

Agreguje elementy zbioru wykorzystując podaną funkcję. Funkcja redukująca musi być asocjacyjna [(a x b) x c = a x (b x c)] i przemienna [a x b = b x a].

In [114]:
RDDlist.reduce(lambda x,y: x+y)

45

In [115]:
RDDlist2.reduce(lambda x,y: x+y)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

##### aggregate(zeroValue, seqOp, combOp)

Działa podobnie do `aggregateByKey`

In [116]:
RDDother.aggregate(0, (lambda acc,x: acc + int(x)), (lambda acc1,acc2: acc1 + acc2))

45

> **TODO**: Po odfiltrowaniu "0" oblicz iloczyn wartości RDDother

In [119]:
RDDother.filter(lambda x: x != '0').collect()


['1', '2', '3', '4', '5', '6', '7', '8', '9']

##### fold(zeroValue, op)

Zwraca zagregowane przy pomocy funkcji `op` wartości RDD. Funkcja `op` przyjmuje 2 wartości i musi być asocjacyjna _[(a x b) x c = a x (b x c)]_.

In [ ]:
RDDlist.fold(0, (lambda acc,x: acc + x))

> **TODO**: Po odfiltrowaniu 0 i liczb nieparzystych oblicz iloczyn wartości RDDlist

##### max(), mean(), min(), stdev(), variance(), stats()

In [120]:
RDDlist.max()

9

In [121]:
RDDlist.mean()

4.5

In [122]:
RDDlist.min()

0

In [123]:
RDDlist.stdev()

2.8722813232690143

In [124]:
RDDlist.variance()

8.25

In [125]:
RDDlist.stats()

(count: 10, mean: 4.5, stdev: 2.8722813232690143, max: 9.0, min: 0.0)

##### reduceByKeyLocally(func)

Działa analogicznie do `reduceByKey` z tym, że zwraca wynik do drivera. 

In [126]:
RDDpair4.reduceByKeyLocally(lambda x,y: x+y)

{'A': 13, 'B': 15}